# Modelleringsdel för kunskapskontroll 2

Not: All visualisering, EDA, Confusion Matrix, etc. och diverse experimenterande finns i notebook MNIST-modellering - experiment. 

Detta är bara modellering med utvärdering av olika modeller, val av bäst modell, omträning på hela datasetet
och sedan dump ned till en ny joblib.


## Steg 1. Importera nödvändiga paket.

In [1]:
# Av eget intresse vill jag gärna veta hur lång tid olika saker tar.
import time
notebook_start = time.time()  
t0 = time.time()

# Paket för datahantering
import numpy as np
import pandas as pd

# Dataset och modeller
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

# Preprocessing/pipelin
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Modeller 
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier

# Bonusmodeller på slutet
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier 
    
# För export av modellen/scalern för vidare användning i Streamlit-appen
import joblib

cell_time = time.time() - t0
total_time = time.time() - notebook_start
mins, secs = divmod(total_time, 60)

print(f">>> Tid för denna cell: {cell_time:.1f} sekunder")
print(f">>> Total tid sedan start: {int(mins)} minuter och {int(secs)} sekunder")

>>> Tid för denna cell: 1.8 sekunder
>>> Total tid sedan start: 0 minuter och 1 sekunder


## Steg 2. Läs in MNISt-datasetet och splitta det. 
*//Jag är en visare man nu och splittar direkt//*

In [2]:
t0 = time.time()

# Läs in hela MNIST
mnist = fetch_openml('mnist_784', version=1, cache=True, as_frame=False, parser='auto')
X = mnist["data"]              
y = mnist["target"].astype(np.uint8)

# Splittar 80/20 för träning (och korsvalidering) respektive test
# Stratifiering garanterar statistisk säkerhet med jämn fördelning av siffrorna i båda seten
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

# För att snabba upp hanteringen och slippa köra StandardScaler normaliserar vi data
# Genom att dela med 255 hamnar allt mellan 0 och 1 som float64.
X_train = X_train / 255.0
X_test = X_test / 255.0


print(f"Träningsset: {X_train.shape}")
print(f"Testset: {X_test.shape}")

cell_time = time.time() - t0
total_time = time.time() - notebook_start
mins, secs = divmod(total_time, 60)

print(f">>> Tid för denna cell: {cell_time:.1f} sekunder")
print(f">>> Total tid sedan start: {int(mins)} minuter och {int(secs)} sekunder")

Träningsset: (56000, 784)
Testset: (14000, 784)
>>> Tid för denna cell: 3.2 sekunder
>>> Total tid sedan start: 0 minuter och 5 sekunder


## Steg 3. Dags att börja testa olika modeller. Jag har till att börja med valt en linjär modell, en icke-linjär modell och en trädmodell. 

Vi börjar med att skapa en dict för resultaten

In [3]:
results = {}

Linjär modell: SGDClassifier

## SGDClassifier

In [4]:
t0 = time.time()

# Pipeline
sgd_pipe = Pipeline([
    ('sgd', SGDClassifier(random_state=42))
])

# Param_grid: log_loss ger en logistisk regression som "baseline". 
# Vi ser vilken Alpha som ger bäst regularisering 
param_grid_sgd = {
    'sgd__loss': ['log_loss'],
    'sgd__alpha': [0.0001, 0.01],
    'sgd__max_iter': [1000]
}

# GridSearchCV: CV=3 ger bra balans mellan tid och statistisk säkerhet
sgd_grid = GridSearchCV(
    estimator=sgd_pipe,
    param_grid=param_grid_sgd,
    cv=3,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

# Träning av modellen 
sgd_grid.fit(X_train, y_train)

# Resultat
print(f"Bästa parametrar: {sgd_grid.best_params_}")
print(f"Bästa genomsnittliga CV-accuracy: {sgd_grid.best_score_:.4f}")

# Spara resultaten
results["SGD (Baseline)"] = {
    "Best Accuracy": round(sgd_grid.best_score_, 4),
    "Best Params": sgd_grid.best_params_
}

cell_time = time.time() - t0
total_time = time.time() - notebook_start
mins, secs = divmod(total_time, 60)

print(f">>> Tid för denna cell: {cell_time:.1f} sekunder")
print(f">>> Total tid sedan start: {int(mins)} minuter och {int(secs)} sekunder")

Fitting 3 folds for each of 2 candidates, totalling 6 fits
Bästa parametrar: {'sgd__alpha': 0.0001, 'sgd__loss': 'log_loss', 'sgd__max_iter': 1000}
Bästa genomsnittliga CV-accuracy: 0.9110
>>> Tid för denna cell: 25.0 sekunder
>>> Total tid sedan start: 0 minuter och 30 sekunder


En icke-linjär modell. SVC har jag hört gott om. 

## SVC

In [5]:
t0 = time.time()

# Den här modellen tog lång tid. Jag uppskattade dess kvalitet med ett mindre urval först 
# och körde ett par olika omgångar för att hitta bästa hyperparametrarna innan jag körde
# på hela datasetet med de optimala hyperparametrar.

# Borttagen "sub-sampling"
"""X_train_svc_sub, _, y_train_svc_sub, _ = train_test_split(  
    X_train, y_train, train_size=20000, stratify=y_train, random_state=42
)
"""

# Pipeline
svc_pipe = Pipeline([
    ('svc', SVC(kernel='rbf', random_state=42))
])

# Param_grid
param_grid_svc = {
    'svc__C': [10],
    'svc__gamma': ['scale']
}

# GridSearchCV
svc_grid = GridSearchCV(
    estimator=svc_pipe,
    param_grid=param_grid_svc,
    cv=3,
    scoring='accuracy',
    n_jobs=-1,
    # verbose=2  Tog bort detta eftersom jag bara vill ha ut resultatet
)

# Träning 
svc_grid.fit(X_train, y_train)

# Resultat
print(f"Bästa SVC-parametrar: {svc_grid.best_params_}")
print(f"Bästa SVC CV-accuracy: {svc_grid.best_score_:.4f}")

results["SVC (Non-linear - higher C)"] = {
    "Best Accuracy": round(svc_grid.best_score_, 4),
    "Best Params": svc_grid.best_params_
}

cell_time = time.time() - t0
total_time = time.time() - notebook_start
mins, secs = divmod(total_time, 60)

print(f">>> Tid för denna cell: {cell_time:.1f} sekunder")
print(f">>> Total tid sedan start: {int(mins)} minuter och {int(secs)} sekunder")

Bästa SVC-parametrar: {'svc__C': 10, 'svc__gamma': 'scale'}
Bästa SVC CV-accuracy: 0.9810
>>> Tid för denna cell: 296.5 sekunder
>>> Total tid sedan start: 5 minuter och 26 sekunder


En trädmodell - Extra Trees fick bra resultat i boken så den kör vi med.

## Extra Trees

In [6]:
t0 = time.time()

# Pipeline
extra_trees_clf = ExtraTreesClassifier(random_state=42)

pipeline = Pipeline([
    ("model", extra_trees_clf)
])

# Param_grid
param_grid = {
    "model__n_estimators": [200, 250, 300]
}

# GridSearchCV
extra_trees_grid = GridSearchCV(
    pipeline,
    param_grid=param_grid,
    cv=3,
    scoring="accuracy",
    n_jobs=-1,  
    verbose=1   
)

# Träning
extra_trees_grid.fit(X_train, y_train)

# Spara resultat
extra_trees_accuracy = extra_trees_grid.best_score_
extra_trees_params = extra_trees_grid.best_params_

results["Extra Trees"] = {
    "Best Accuracy": round(extra_trees_accuracy, 4),
    "Best Params": extra_trees_params
}

print(f"Bästa parametrar: {extra_trees_params}")
print(f"Bästa CV-accuracy: {extra_trees_accuracy:.4f}")

cell_time = time.time() - t0
total_time = time.time() - notebook_start
mins, secs = divmod(total_time, 60)

print(f">>> Tid för denna cell: {cell_time:.1f} sekunder")
print(f">>> Total tid sedan start: {int(mins)} minuter och {int(secs)} sekunder")

Fitting 3 folds for each of 3 candidates, totalling 9 fits
Bästa parametrar: {'model__n_estimators': 300}
Bästa CV-accuracy: 0.9700
>>> Tid för denna cell: 133.7 sekunder
>>> Total tid sedan start: 7 minuter och 40 sekunder


Innan jag ger mig så provar jag två modeller till: XGBoost och dess "lättviktskompis" LightGBM som kanske kan ge en liten vassare accuracy med sin leaf-konstruktion.

## XGBClassifier

In [7]:
t0 = time.time()

xgb_clf = XGBClassifier(
    random_state=42, 
    eval_metric='mlogloss'
)

# Param_grid
param_grid_xgb = {
    'n_estimators': [100, 200],
    'learning_rate': [0.1],
    'max_depth': [6]
}

# GridSearchCV
xgb_grid = GridSearchCV(
    estimator=xgb_clf,
    param_grid=param_grid_xgb,
    cv=3,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

# Träning
xgb_grid.fit(X_train, y_train)

# Spara resultat
results["XGBoost"] = {
    "Best Accuracy": round(xgb_grid.best_score_, 4),
    "Best Params": xgb_grid.best_params_
}

print(f"Bästa XGBoost-parametrar: {xgb_grid.best_params_}")
print(f"Bästa XGBoost CV-accuracy: {xgb_grid.best_score_:.4f}")

cell_time = time.time() - t0
total_time = time.time() - notebook_start
mins, secs = divmod(total_time, 60)

print(f">>> Tid för denna cell: {cell_time:.1f} sekunder")
print(f">>> Total tid sedan start: {int(mins)} minuter och {int(secs)} sekunder")

Fitting 3 folds for each of 2 candidates, totalling 6 fits
Bästa XGBoost-parametrar: {'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 200}
Bästa XGBoost CV-accuracy: 0.9722
>>> Tid för denna cell: 549.6 sekunder
>>> Total tid sedan start: 16 minuter och 49 sekunder


## LightGBM

In [8]:
t0 = time.time()

lgbm_clf = LGBMClassifier(
    random_state=42, 
    n_jobs=-1, 
    verbosity=-1
)

# Param_grid
param_grid_lgbm = {
    'n_estimators': [100, 200],
    'learning_rate': [0.1],
    'num_leaves': [31, 60] 
}

# GridSearchCV
lgbm_grid = GridSearchCV(
    lgbm_clf, 
    param_grid_lgbm, 
    cv=3, 
    scoring='accuracy', 
    n_jobs=-1, 
    verbose=1
)

# Träning
lgbm_grid.fit(X_train, y_train)

# Spara resultat
results["LightGBM"] = {
    "Best Accuracy": round(lgbm_grid.best_score_, 4),
    "Best Params": lgbm_grid.best_params_
}

print(f"Bästa GBMLight-parametrar: {lgbm_grid.best_params_}")
print(f"Bästa GBMLight-accuracy: {lgbm_grid.best_score_:.4f}")

cell_time = time.time() - t0
total_time = time.time() - notebook_start
mins, secs = divmod(total_time, 60)

print(f">>> Tid för denna cell: {cell_time:.1f} sekunder")
print(f">>> Total tid sedan start: {int(mins)} minuter och {int(secs)} sekunder")

Fitting 3 folds for each of 4 candidates, totalling 12 fits
Bästa GBMLight-parametrar: {'learning_rate': 0.1, 'n_estimators': 200, 'num_leaves': 31}
Bästa GBMLight-accuracy: 0.9776
>>> Tid för denna cell: 476.3 sekunder
>>> Total tid sedan start: 24 minuter och 46 sekunder


Så tar vi en titt på resultaten för att se vilken modell det ska bli.

In [9]:
# Vi får snygga till min dict
results_df = pd.DataFrame(results).T

# Sortera på Accuracy så vi ser vilken som var bäst
results_df = results_df.sort_values(by="Best Accuracy", ascending=False)

# Visa tabellen 
print("Jämförelse av olika modeller")
display(results_df)

Jämförelse av olika modeller


,Best Accuracy,Best Params
SVC (Non-linear - higher C),0.981,"{'svc__C': 10, 'svc__gamma': 'scale'}"
LightGBM,0.9776,"{'learning_rate': 0.1, 'n_estimators': 200, 'n..."
XGBoost,0.9722,"{'learning_rate': 0.1, 'max_depth': 6, 'n_esti..."
Extra Trees,0.97,{'model__n_estimators': 300}
SGD (Baseline),0.911,"{'sgd__alpha': 0.0001, 'sgd__loss': 'log_loss'..."


Min magkänsla var att LightGBM skulle vara mest effektiv tack vare leaf-wise-strukturen. När jag tog med alla träningsdata visade det sig dock att SVC inte bara var mer pricksäker, utan även snabbare.

Då ser vi hur den presterar på mitt testset. 

In [10]:
t0 = time.time()

best_svc = svc_grid.best_estimator_

# Utvärdera på testdata (X_test, y_test)
test_accuracy = best_svc.score(X_test, y_test)

print(f"--- SLUTGILTIG UTVÄRDERING ---")
print(f"Modell: SVC")
print(f"Bästa parametrar: {svc_grid.best_params_}")
print(f"Accuracy på testdata: {test_accuracy:.4f}")

cell_time = time.time() - t0
total_time = time.time() - notebook_start
mins, secs = divmod(total_time, 60)

print(f">>> Tid för denna cell: {cell_time:.1f} sekunder")
print(f">>> Total tid sedan start: {int(mins)} minuter och {int(secs)} sekunder")

--- SLUTGILTIG UTVÄRDERING ---
Modell: SVC
Bästa parametrar: {'svc__C': 10, 'svc__gamma': 'scale'}
Accuracy på testdata: 0.9834
>>> Tid för denna cell: 55.0 sekunder
>>> Total tid sedan start: 25 minuter och 41 sekunder


0.9834 - över 98 procent alltså. Det får nog räknas som godkänt tycker jag. Dags att träna modellen på *alla* data.

In [11]:
t0 = time.time()

# Slå ihop tränings- och testdata
X_all = np.concatenate((X_train, X_test))
y_all = np.concatenate((y_train, y_test))

# Hämta parametrarna från min grid
final_svc_model = svc_grid.best_estimator_

print("Tränar modellen. Please wait (det här kan ta ett tag ... :D )")
final_svc_model.fit(X_all, y_all)
print("Sådär. Modellen är nu färdig för att börja prediktera siffror.")

cell_time = time.time() - t0
total_time = time.time() - notebook_start
mins, secs = divmod(total_time, 60)

print(f">>> Tid för denna cell: {cell_time:.1f} sekunder")
print(f">>> Total tid sedan start: {int(mins)} minuter och {int(secs)} sekunder")

Tränar modellen. Please wait (det här kan ta ett tag ... :D )
Sådär. Modellen är nu färdig för att börja prediktera siffror.
>>> Tid för denna cell: 130.5 sekunder
>>> Total tid sedan start: 27 minuter och 51 sekunder


Vi lagrar ned den här modellen via joblib så vi kan anropa den från Streamlit

In [12]:
t0 = time.time()

model_filename = 'mnist_model_final_svc.joblib'

# Spara modellen
joblib.dump(final_svc_model, model_filename)

print("Modellen är sparad via joblib! Nu kör vi!!!")

cell_time = time.time() - t0
total_time = time.time() - notebook_start
mins, secs = divmod(total_time, 60)

print(f">>> Tid för denna cell: {cell_time:.1f} sekunder")
print(f">>> Total tid sedan start: {int(mins)} minuter och {int(secs)} sekunder")

Modellen är sparad via joblib! Nu kör vi!!!
>>> Tid för denna cell: 0.0 sekunder
>>> Total tid sedan start: 27 minuter och 51 sekunder
